# SQL Exercise in November 2022

__Date: 04/11/2022__
### [175. Combine Two Tables](https://leetcode.com/problems/combine-two-tables/)

```mysql
SELECT
  firstName,
  lastName,
  city,
  state
FROM
  Person
  LEFT JOIN Address ON Person.personId = Address.personId
```

__Date: 07/11/2022__
### [1393. Capital Gain/Loss](https://leetcode.com/problems/capital-gainloss/)

```mysql
SELECT 
  stock_name, 
  SUM(
    CASE WHEN operation = 'Buy' THEN - price ELSE price END
  ) capital_gain_loss 
FROM 
  Stocks 
GROUP BY 
  stock_name
```

__Date: 08/11/2022__
### [2388. Change Null Values in a Table to the Previous Value](https://leetcode.com/problems/change-null-values-in-a-table-to-the-previous-value/)

```mysql
SELECT
  id,
  IFNULL(drink, SubTable_drink) AS drink
FROM
  (
    SELECT
      WithRowNum.id,
      WithRowNum.drink,
      SubTable.drink AS SubTable_drink
    FROM
      (
        SELECT
          ROW_NUMBER() OVER() AS row_num,
          id,
          drink
        FROM
          CoffeeShop
      ) AS WithRowNum
      LEFT JOIN (
        SELECT
          ROW_NUMBER() OVER() AS row_num,
          id,
          drink
        FROM
          CoffeeShop
      ) AS SubTable ON (
        WithRowNum.drink IS NULL
        AND WithRowNum.row_num > SubTable.row_num
        AND SubTable.drink IS NOT NULL
      )
    GROUP BY
      WithRowNum.id
  ) AS Result
```
***

```mysql
WITH cte AS (SELECT id, drink, ROW_NUMBER() OVER () AS nb FROM CoffeeShop), -- nb = initial row order
     cte2 AS (SELECT id, drink, nb, SUM(1- (CASE WHEN drink IS NULL THEN 1 ELSE 0 END)) OVER (ORDER BY nb) AS group_id FROM cte) -- same group_id -> same drink
SELECT id, FIRST_VALUE(drink) OVER (PARTITION BY group_id) AS drink
FROM cte2
ORDER BY nb
```

__Date: 09/11/2022__
### [626. Exchange Seats](https://leetcode.com/problems/exchange-seats/)

```mysql
SELECT 
  CASE WHEN id % 2 = 1 
  AND id != (
    SELECT 
      MAX(id) 
    FROM 
      seat
  ) THEN id + 1 WHEN id % 2 = 0 THEN id - 1 ELSE id END AS id, 
  student 
FROM 
  seat 
ORDER BY 
  id
```

__Date: 10/11/2022__
### [1398. Customers Who Bought Products A and B but Not C](https://leetcode.com/problems/customers-who-bought-products-a-and-b-but-not-c/)

```mysql
SELECT
  c.customer_id,
  c.customer_name
FROM
  Customers c
  INNER JOIN Orders o ON c.customer_id = o.customer_id
GROUP BY
  c.customer_id
HAVING
  SUM(o.product_name = 'C') = 0
  AND SUM(o.product_name = 'A') > 0
  AND SUM(o.product_name = 'B') > 0
ORDER BY
  c.customer_id
```
***


```mysql
SELECT
  c.customer_id,
  c.customer_name
FROM
  Customers c
  JOIN Orders a ON a.customer_id = c.customer_id
  AND a.product_name = "A"
  JOIN Orders b ON b.customer_id = c.customer_id
  AND b.product_name = "B"
  LEFT OUTER JOIN Orders o on o.customer_id = c.customer_id
  AND o.product_name = "C"
WHERE
  o.order_id IS NULL
```
***


```mysql
SELECT
  DISTINCT b.customer_id,
  b.customer_name
FROM
  Orders a
  JOIN Customers b ON a.customer_id = b.customer_id
WHERE
  a.customer_id NOT IN (
    select
      customer_id
    FROM
      Orders
    WHERE
      product_name = 'C'
  )
  AND a.customer_id IN (
    SELECT
      customer_id
    FROM
      Orders
    WHERE
      product_name = 'A'
  )
  AND a.customer_id IN (
    SELECT
      customer_id
    FROM
      Orders
    WHERE
      product_name = 'B'
  )
ORDER BY
  1
```

__Date: 11/11/2022__
### [1667. Fix Names in a Table](https://leetcode.com/problems/fix-names-in-a-table/)

```mysql
SELECT
  user_id,
  CONCAT(
    UPPER(SUBSTRING(name, 1, 1)),
    LOWER(SUBSTRING(name, 2))
  ) AS name
FROM
  Users
ORDER BY
  user_id
```

__Date: 14/11/2022__
### [1715. Count Apples and Oranges](https://leetcode.com/problems/count-apples-and-oranges/)

```mysql
SELECT 
  SUM(
    CASE WHEN b.chest_id IS NOT NULL THEN (b.apple_count + c.apple_count) ELSE b.apple_count END
  ) AS apple_count, 
  SUM(
    CASE WHEN b.chest_id IS NOT NULL THEN (b.orange_count + c.orange_count) ELSE b.orange_count END
  ) AS orange_count 
FROM 
  boxes b 
  LEFT JOIN chests c ON b.chest_id = c.chest_id
```

__Date: 16/11/2022__
### [178. Rank Scores](https://leetcode.com/problems/rank-scores/)

```mysql
SELECT 
  score, 
  dense_rank() OVER(
    ORDER BY 
      score DESC
  ) as "rank" 
FROM 
  Scores
```
***
```mysql
SELECT 
  S1.score, 
  (
    SELECT 
      COUNT(DISTINCT Score) 
    FROM 
      Scores 
    WHERE 
      score >= S1.Score
  ) AS "rank" 
FROM 
  Scores S1 
ORDER BY 
  S1.score DESC
```

__Date: 17/11/2022__
### [1158. Market Analysis I](https://leetcode.com/problems/market-analysis-i/)

```mysql
SELECT u.user_id AS buyer_id, join_date, COUNT(order_date) AS orders_in_2019 
FROM Users as u
LEFT JOIN Orders as o
ON u.user_id = o.buyer_id
AND YEAR(order_date) = 2019
GROUP BY u.user_id
```